In [1]:
import json
import statham
from json_ref_dict import materialize, RefDict
from statham.schema.parser import parse
from statham.titles import title_labeller
from statham.serializers.python import serialize_python

from typing import Set, Type, Union

from statham.schema.elements import Element, Object
from statham.schema.elements.meta import ObjectMeta
from statham.serializers.orderer import orderer, get_children

In [2]:
# with open('schemata.json') as json_file:
#     schemata = json.load(json_file)

In [3]:
schemata = materialize(
    RefDict('schemata.json'), context_labeller=title_labeller()
)

In [4]:
elements = parse(schemata)

In [5]:
def _get_imports(declarations: str, *elements: Element) -> str:
    """Get import statements required by the elements."""
    imports = [
        _get_standard_imports(declarations),
        _get_statham_imports(declarations, *elements),
    ]
    return "\n\n".join(block for block in imports if block)

def _get_standard_imports(declaration: str) -> str:
    """Get group if imports from standard library.

    Currently only includes type annotations.
    """
    type_imports = ", ".join(
        [
            annotation
            for annotation in ("Any", "List", "Union")
            if annotation in declaration
        ]
    )
    if not type_imports:
        return ""
    return f"from typing import {type_imports}"


def _get_statham_imports(declaration: str, *elements: Element) -> str:
    """Construct imports from statham submodules."""
    statham_imports = []
    if "Maybe" in declaration:
        statham_imports.append("from statham.schema.constants import Maybe")
    element_imports = _get_element_imports(*elements)
    if element_imports:
        statham_imports.append(element_imports)
    if "Property" in declaration:
        statham_imports.append("from statham.schema.property import Property")
    return "\n".join(statham_imports)


def _get_element_imports(*elements: Element) -> str:
    """Get the import string for the elements in use."""
    prefix = "from statham.schema.elements import "
    max_length = 80
    import_names = [
        elem_type.__name__
        for elem_type in set.union(
            *(_get_single_element_imports(element) for element in elements)
        )
    ]
    if not import_names:
        return ""
    imports = ", ".join(sorted(import_names))
    if max_length < len(prefix) + len(imports):
        imports = "\n    ".join(["("] + imports.split(" ")) + ",\n)"
    return prefix + imports


def _get_single_element_imports(
    element: Element,
) -> Set[Union[Type[Element], ObjectMeta]]:
    """Extract the set of element types used by a given element."""
    get_type = (
        lambda elem: Object if isinstance(elem, ObjectMeta) else type(elem)
    )
    children = [element, *get_children(element)]
    return set(map(get_type, children))  # type: ignore

In [6]:
import inspect
from typing import Dict, cast
from statham.schema.constants import NotPassed
from statham.schema.property import _Property
from statham.schema.elements import Boolean, Number, String

def format_item(item):
    if isinstance(item, list):
        return str([format_item(i) for i in item])
    if isinstance(item, int) or isinstance(item, float):
        return str(item)
    elif isinstance(item, str):
        item = item.replace('"', '\\"')
        return f'"{item}"'
    else:
        raise ValueError(f'type of item {repr(item)} is {type(item).__name__}')

def annotation(prop):
    is_required = prop.required or not isinstance(
        getattr(prop.element, "default", NotPassed()), NotPassed
    )
    prop_enum = getattr(prop.element, "enum", NotPassed())
    if not isinstance(prop_enum, NotPassed):
        annotation = f"Literal[{','.join([format_item(e) for e in prop_enum])}]"
    else:
        annotation = prop.element.annotation

    if is_required:
        return annotation
    else:
        return f"Optional[{annotation}]"

def repr_property(prop: _Property) -> str:
    string = ''
    default = getattr(prop.element, "default", NotPassed())
    description = getattr(prop.element, "description", NotPassed())
    enumvals = getattr(prop.element, "enum", NotPassed())

    if not isinstance(description, NotPassed):
         string += f"    '''\n    {description}\n    '''\n"

    if default is not None and not isinstance(default, NotPassed):
         string += f"    {prop.name}: {annotation(prop)} = {format_item(default)}\n"
    else:
         string += f"    {prop.name}: {annotation(prop)}\n"

    return string

def to_dataclass(cls) -> str:
    class_def = f'@dataclass\nclass {repr(cls)}():\n'
    if not cls.description is None and not isinstance(
        cls.description, NotPassed
    ):
        class_def += f'    """{cls.description}"""\n'
    if not cls.properties:
        class_def += f'    pass\n'
        
    for property_ in cast(
        Dict[str, _Property], cls.properties or {}
    ).values():
        class_def += repr_property(property_)
    return class_def

def serialize_dataclasses(*elements: Element) -> str:
    """Serialize schema elements to python declaration string.

    Captures declaration of the first Object elements, and any subsequent
    elements this depends on. Module imports and declaration order are
    dynamically inferred.

    :param elements: The :class:`~statham.schema.elements.Element` objects
        to serialize.
    :return: Python module contents as a string, declaring the element tree.
    """
    declarations = "\n\n".join(
        [to_dataclass(object_model) for object_model in orderer(*elements)]
    )
    imports = "\n".join([
        'from dataclasses import dataclass',
        'from typing import Any, List, Union, Optional, Literal'
    ])
    return "\n\n\n".join(block for block in [imports, declarations] if block)

In [7]:
with open("covfee_dataclasses.py", "w") as fh:
    fh.write(serialize_dataclasses(*elements))